<a href="https://colab.research.google.com/github/madynamics/Noise-modelling/blob/main/Diabetic_Retinopathy_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Download data from Kaggle or Google Drive

In [ ]:
## DO NOT START HERE, START FROM '# 0.get the data' below

!pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir /root/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle/kaggle.json /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json
! kaggle competitions download -c diabetic-retinopathy-detection

In [ ]:
! unzip /content/diabetic-retinopathy-detection.zip

In [ ]:
!rm /content/diabetic-retinopathy-detection.zip

In [ ]:
!cat test.zip.* > test.zip

In [ ]:
!rm /content/test.zip.001
!rm /content/test.zip.002
!rm /content/test.zip.003
!rm /content/test.zip.004
!rm /content/test.zip.005
!rm /content/test.zip.006
!rm /content/test.zip.007

In [ ]:
!cat train.zip.* > train.zip

In [ ]:
!rm /content/train.zip.001
!rm /content/train.zip.002
!rm /content/train.zip.003
!rm /content/train.zip.004
!rm /content/train.zip.005

In [ ]:
! unzip /content/test.zip
! unzip /content/train.zip
! unzip /content/trainLabels.csv.zip

In [ ]:
!rm /content/train.zip
!rm /content/test.zip
!rm /content/trainLabels.csv.zip

In [ ]:
!mkdir /content/DiabeticRetinopathyDetection
!mv /content/test/ /content/DiabeticRetinopathyDetection
!mv /content/train/ /content/DiabeticRetinopathyDetection
!mv /content/trainLabels.csv /content/DiabeticRetinopathyDetection

In [ ]:
!mv /content/DiabeticRetinopathyDetection/ /content/drive/MyDrive/

In [ ]:
# 0.get the data

!pip install kaggle
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


1. Data Pre-processing

In [ ]:
## NEED NOT EXECUTE, GO TO NEXT CELL

## pre-processing to enlarge the portion of retina

## takes a lot of time
import os
import numpy as np
from PIL import Image
import warnings
from multiprocessing import Pool
import cv2
from tqdm import tqdm

def trim(im):
    """
    Converts image to grayscale using cv2, then computes binary matrix
    of the pixels that are above a certain threshold, then takes out
    the first row where a certain percetage of the pixels are above the
    threshold will be the first clip point. Same idea for col, max row, max col.
    """
    percentage = 0.02

    img = np.array(im)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    im = img_gray > 0.1 * np.mean(img_gray[img_gray != 0])
    row_sums = np.sum(im, axis=1)
    col_sums = np.sum(im, axis=0)
    rows = np.where(row_sums > img.shape[1] * percentage)[0]
    cols = np.where(col_sums > img.shape[0] * percentage)[0]
    min_row, min_col = np.min(rows), np.min(cols)
    max_row, max_col = np.max(rows), np.max(cols)
    im_crop = img[min_row : max_row + 1, min_col : max_col + 1]
    return Image.fromarray(im_crop)


def resize_maintain_aspect(image, desired_size):
    """
    Stole this from some stackoverflow post but can't remember which,
    this will add padding to maintain the aspect ratio.
    """
    old_size = image.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    im = image.resize(new_size, Image.ANTIALIAS)
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size - new_size[0]) // 2, (desired_size - new_size[1]) // 2))
    return new_im

def save_single(args):
    img_file, input_path_folder, output_path_folder, output_size = args
    if img_file.startswith('.ipynb_checkpoints'):
        return
    input_path = os.path.join(input_path_folder, img_file)
    output_path = os.path.join(output_path_folder, img_file)
    if os.path.exists(output_path):
        return
    image_original = Image.open(input_path)
    image = trim(image_original)
    image = resize_maintain_aspect(image, desired_size=output_size[0])
    image.save(output_path)


def fast_image_resize(input_path_folder, output_path_folder, output_size=None):
    """
    Uses multiprocessing to make it fast
    """
    if not output_size:
        warnings.warn("Need to specify output_size! For example: output_size=100")
        exit()

    if not os.path.exists(output_path_folder):
        os.makedirs(output_path_folder)

    jobs = [
        (file, input_path_folder, output_path_folder, output_size)
        for file in os.listdir(input_path_folder)
    ]

    with Pool() as p:
        list(tqdm(p.imap_unordered(save_single, jobs), total=len(jobs)))

In [ ]:
## NEED NOT EXECUTE, GO TO NEXT CELL

# if you believe the folder may be corrupted, you may use the following line
# !rm -r /content/drive/MyDrive/DiabeticRetinopathyDetection/test_resized

In [ ]:
## NEED NOT EXECUTE, GO TO NEXT CELL
# CHANGE SIZE (ONLY FOR PREPROCESSING)

# fast_image_resize("/content/drive/MyDrive/DiabeticRetinopathyDetection/train/", "/content/drive/MyDrive/DiabeticRetinopathyDetection/train_resized/", output_size=(CROP_W, CROP_H))
fast_image_resize("/content/drive/MyDrive/DiabeticRetinopathyDetection/test/", "/content/drive/MyDrive/DiabeticRetinopathyDetection/test_resized/", output_size=(728, 728))

100%|██████████| 53577/53577 [00:05<00:00, 9732.54it/s]


In [ ]:
# 1.preprocessing
import pandas as pd

trainLabel = pd.read_csv("/content/drive/MyDrive/DiabeticRetinopathyDetection/trainLabels.csv")

trainLabel.iloc[0:1208,:].to_csv("valLabel.csv",index=False)
trainLabel.iloc[1209:,:].to_csv("trainLabel.csv",index=False)

2. Data loader

In [ ]:
# 2. define the dataset

import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from tqdm import tqdm
import os
import numpy as np
import pandas as pd


# Data augmentation for images
train_transforms = A.Compose(
    [
        A.Resize(width=760, height=760),
        A.RandomCrop(height=728, width=728),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Blur(p=0.3),
        A.CLAHE(p=0.3),
        A.ColorJitter(p=0.3),
        A.CoarseDropout(max_holes=12, max_height=20, max_width=20, p=0.3),
        A.Affine(shear=30, rotate=0, p=0.2, mode="constant"),
        A.Normalize(
            mean=[0.3199, 0.2240, 0.1609],
            std=[0.3020, 0.2183, 0.1741],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

val_transforms = A.Compose(
    [
        A.Resize(height=728, width=728),
        A.Normalize(
            mean=[0.3199, 0.2240, 0.1609],
            std=[0.3020, 0.2183, 0.1741],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

class DRDataset(Dataset):
    def __init__(self, images_folder, path_to_csv, train=True, transform=None):
        super().__init__()
        # each row of the csv file contains: 1) image name; 2) label
        self.data = pd.read_csv(path_to_csv)
        self.images_folder = images_folder
        self.image_files = os.listdir(images_folder)
        self.transform = transform
        self.train = train

    def __len__(self):
        return self.data.shape[0] if self.train else len(self.image_files)

    def __getitem__(self, index):
        if self.train:
            image_file, label = self.data.iloc[index]
        else:
            # if test simply return -1 for label, I do this in order to
            # re-use same dataset class for test set submission later on
            image_file, label = self.image_files[index], -1
            image_file = image_file.replace(".jpeg", "")

        # read it with Image.open from PIL to convert it into np.array, because we want to
        # use albumentation
        image = np.array(Image.open(os.path.join(self.images_folder, image_file+".jpeg")))

        # send it to augmentation
        if self.transform:
            # this is the form that albumentation requires
            image = self.transform(image=image)["image"]

        return image, label, image_file

In [ ]:
"""
Test if everything works ok
"""
dataset = DRDataset(
    images_folder="/content/drive/MyDrive/DiabeticRetinopathyDetection/train_resized/",
    path_to_csv="/content/drive/MyDrive/DiabeticRetinopathyDetection/trainLabels.csv",
    transform=val_transforms,
)
loader = DataLoader(
    dataset=dataset, batch_size=32, num_workers=2, shuffle=True, pin_memory=True
)

for x, label, file in tqdm(loader):
    print(x.shape)
    print(label.shape)
    import sys
    sys.exit()

3. Training

In [ ]:
# 3. Define the methods for training

!pip install efficientnet_pytorch

def check_accuracy(loader, model, device="cuda"):
    model.eval()
    all_preds, all_labels = [], []
    num_correct = 0
    num_samples = 0

    for x, y, filename in tqdm(loader):
        x = x.to(device=device)
        y = y.to(device=device)

        with torch.no_grad():
            predictions = model(x)

        # Convert MSE floats to integer predictions
        predictions[predictions < 0.5] = 0
        predictions[(predictions >= 0.5) & (predictions < 1.5)] = 1
        predictions[(predictions >= 1.5) & (predictions < 2.5)] = 2
        predictions[(predictions >= 2.5) & (predictions < 3.5)] = 3
        predictions[(predictions >= 3.5) & (predictions < 100)] = 4
        predictions = predictions.long().view(-1)
        y = y.view(-1)

        num_correct += (predictions == y).sum()
        num_samples += predictions.shape[0]

        # add to lists
        all_preds.append(predictions.detach().cpu().numpy())
        all_labels.append(y.detach().cpu().numpy())
    
    # though the accuracy is printed, it is not important for the game
    print(
        f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}"
    )
    model.train()
    # concatenate the predictions and labels, will be sent to 'cohen_kappa_score' the calculate
    # the score in training
    return np.concatenate(all_preds, axis=0, dtype=np.int64), np.concatenate(
        all_labels, axis=0, dtype=np.int64
    )


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer, lr):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    #optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 4.0 training

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 5e-4
BATCH_SIZE = 32
NUM_EPOCHS = 200
NUM_WORKERS = 6
PIN_MEMORY = True
SAVE_MODEL = True
LOAD_MODEL = True

from torch import nn, optim
from efficientnet_pytorch import EfficientNet


train_ds = DRDataset(
    images_folder="/content/drive/MyDrive/DiabeticRetinopathyDetection/train_resized",
    path_to_csv="/content/trainLabel.csv",
    transform=val_transforms,
)
val_ds = DRDataset(
    images_folder="/content/drive/MyDrive/DiabeticRetinopathyDetection/train_resized",
    path_to_csv="/content/valLabel.csv",
    transform=val_transforms,
)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    num_workers=2,
    pin_memory=PIN_MEMORY,
    shuffle=False,
)
loss_fn = nn.MSELoss()

model = EfficientNet.from_pretrained("efficientnet-b3")
model._fc = nn.Linear(1536, 1)
model = model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler()

def train_one_epoch(loader, model, optimizer, loss_fn, scaler, device):
    losses = []
    loop = tqdm(loader)
    for batch_idx, (data, targets, _) in enumerate(loop):

        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = loss_fn(scores, targets.unsqueeze(1).float())

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

    print(f"Loss average over epoch: {sum(losses)/len(losses)}")

Loaded pretrained weights for efficientnet-b3


In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
# 4.1 training
for epoch in range(NUM_EPOCHS):
    train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)

    # get on validation
    preds, labels = check_accuracy(val_loader, model, DEVICE)
    print(epoch)
    print(f"QuadraticWeightedKappa (Validation): {cohen_kappa_score(labels, preds, weights='quadratic')}")

    if SAVE_MODEL:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint, filename=f"b3_{epoch}.pth.tar")

100%|██████████| 1060/1060 [10:59<00:00,  1.61it/s, loss=0.34]


Loss average over epoch: 0.5541901853708726


100%|██████████| 38/38 [00:16<00:00,  2.36it/s]


Got 882 / 1208 with accuracy 73.01
0
QuadraticWeightedKappa (Validation): 0.7298800671154084
=> Saving checkpoint


100%|██████████| 1060/1060 [09:51<00:00,  1.79it/s, loss=0.224]


...

Loss average over epoch: 0.09623268037190977


100%|██████████| 38/38 [00:15<00:00,  2.40it/s]


Got 936 / 1208 with accuracy 77.48
10
QuadraticWeightedKappa (Validation): 0.7667025719699266
=> Saving checkpoint


  3%|▎         | 37/1060 [00:22<10:32,  1.62it/s, loss=0.0349]


KeyboardInterrupt: ignored

In [ ]:
!mv /content/b3_9.pth.tar /content/drive/MyDrive/DiabeticRetinopathyDetection/

In [ ]:
LEARNING_RATE = 3e-6

for epoch in range(NUM_EPOCHS):
    train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)

    # get on validation
    preds, labels = check_accuracy(val_loader, model, DEVICE)
    print(epoch)
    print(f"QuadraticWeightedKappa (Validation): {cohen_kappa_score(labels, preds, weights='quadratic')}")

    if SAVE_MODEL:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint, filename=f"b3_{epoch}.pth.tar")

100%|██████████| 1060/1060 [09:51<00:00,  1.79it/s, loss=0.0903]


Loss average over epoch: 0.3942960482161


100%|██████████| 38/38 [00:15<00:00,  2.39it/s]


Got 911 / 1208 with accuracy 75.41
0
QuadraticWeightedKappa (Validation): 0.7546569582117789
=> Saving checkpoint


100%|██████████| 1060/1060 [09:51<00:00,  1.79it/s, loss=0.0595]


...

Loss average over epoch: 0.11825784973554172


100%|██████████| 38/38 [00:16<00:00,  2.35it/s]


Got 952 / 1208 with accuracy 78.81
2
QuadraticWeightedKappa (Validation): 0.7707778107183935
=> Saving checkpoint


 30%|███       | 323/1060 [03:01<06:49,  1.80it/s, loss=0.0591]<ipython-input-2-bb891cd5760e>:72: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'JpegImageFile'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  image = np.array(Image.open(os.path.join(self.images_folder, image_file+".jpeg")))
 30%|███       | 323/1060 [03:01<06:55,  1.78it/s, loss=0.0591]


KeyboardInterrupt: ignored

In [ ]:
!cp /content/drive/MyDrive/DiabeticRetinopathyDetection/b3_9.pth.tar /content/

4. Make submission

In [ ]:
# get result

CHECKPOINT_FILE = "b3_9.pth.tar"

if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
    load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer, LEARNING_RATE)

test_ds = DRDataset(
    images_folder="/content/drive/MyDrive/DiabeticRetinopathyDetection/test_resized",
    path_to_csv="/content/trainLabel.csv", # we are not using it
    transform=val_transforms,
    train=False,
)

BATCH_SIZE_Test = 32

test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE_Test, num_workers=6, shuffle=False
)

def make_prediction(model, loader, output_csv="submission.csv"):
    preds = []
    filenames = []
    model.eval()

    for x, y, files in tqdm(loader):
        x = x.to(DEVICE)
        with torch.no_grad():
            predictions = model(x)
            # Convert MSE floats to integer predictions
            predictions[predictions < 0.5] = 0
            predictions[(predictions >= 0.5) & (predictions < 1.5)] = 1
            predictions[(predictions >= 1.5) & (predictions < 2.5)] = 2
            predictions[(predictions >= 2.5) & (predictions < 3.5)] = 3
            predictions[(predictions >= 3.5) & (predictions < 10000000)] = 4
            predictions = predictions.long().squeeze(1)
            preds.append(predictions.cpu().numpy())
            filenames += files

    # concatenate the filename with the predicted label
    df = pd.DataFrame({"image": filenames, "level": np.concatenate(preds, axis=0)})
    df.to_csv(output_csv, index=False)
    model.train()
    print("Done with predictions")

make_prediction(model, test_loader, "submission_.csv")

=> Loading checkpoint


100%|██████████| 1675/1675 [08:34<00:00,  3.26it/s]


Done with predictions


In [ ]:
!cp /content/drive/MyDrive/DiabeticRetinopathyDetection/b3_18.pth.tar /content/

5. Combine data from both eyes
(Did not use for final submission)

In [ ]:
# 5. more tricks

import torch.nn.functional as F
import warnings

CHECKPOINT_FILE = "b3_9.pth.tar"

if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
    load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer, LEARNING_RATE)

def get_csv_for_blend(loader, model, output_csv_file):
    warnings.warn("Important to have shuffle=False (and to ensure batch size is even size) when running get_csv_for_blend also set val_transforms to train_loader!")
    model.eval()
    filename_first = []
    filename_second = []
    labels_first = []
    labels_second = []
    all_features = []

    for idx, (images, y, image_files) in enumerate(tqdm(loader)):
        images = images.to(DEVICE)

        with torch.no_grad():
            features = F.adaptive_avg_pool2d(
                model.extract_features(images), output_size=1
            )
            features_logits = features.reshape(features.shape[0] // 2, 2, features.shape[1])
            preds = model(images).reshape(images.shape[0] // 2, 2, 1)
            new_features = (
                torch.cat([features_logits, preds], dim=2)
                .view(preds.shape[0], -1)
                .cpu()
                .numpy()
            )
            all_features.append(new_features)
            filename_first += image_files[::2]
            filename_second += image_files[1::2]
            labels_first.append(y[::2].cpu().numpy())
            labels_second.append(y[1::2].cpu().numpy())

    all_features = np.concatenate(all_features, axis=0)
    df = pd.DataFrame(
        data=all_features, columns=[f"f_{idx}" for idx in range(all_features.shape[1])]
    )
    df["label_first"] = np.concatenate(labels_first, axis=0)
    df["label_second"] = np.concatenate(labels_second, axis=0)
    df["file_first"] = filename_first
    df["file_second"] = filename_second
    df.to_csv(output_csv_file, index=False)
    model.train()

train_ds = DRDataset(
    images_folder="/content/drive/MyDrive/DiabeticRetinopathyDetection/train_resized",
    path_to_csv="/content/trainLabel.csv",
    transform=val_transforms,
)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
)

# get_csv_for_blend(val_loader, model, "/content/drive/MyDrive/DiabeticRetinopathyDetection/val_blend.csv")
get_csv_for_blend(train_loader, model, "/content/drive/MyDrive/DiabeticRetinopathyDetection/train_blend.csv")
get_csv_for_blend(test_loader, model, "/content/drive/MyDrive/DiabeticRetinopathyDetection/test_blend.csv")

def make_prediction(model, loader, file):
    preds = []
    filenames = []
    model.eval()

    for x, y, files in tqdm(loader):
        x = x.to(DEVICE)
        with torch.no_grad():
            predictions = model(x)
            # Convert MSE floats to integer predictions
            predictions[predictions < 0.5] = 0
            predictions[(predictions >= 0.5) & (predictions < 1.5)] = 1
            predictions[(predictions >= 1.5) & (predictions < 2.5)] = 2
            predictions[(predictions >= 2.5) & (predictions < 3.5)] = 3
            predictions[(predictions >= 3.5) & (predictions < 100)] = 4
            predictions = predictions.long().view(-1)
            y = y.view(-1)

            preds.append(predictions.cpu().numpy())
            filenames += map(list, zip(files[0], files[1]))

    filenames = [item for sublist in filenames for item in sublist]
    df = pd.DataFrame({"image": filenames, "level": np.concatenate(preds, axis=0)})
    df.to_csv(file, index=False)
    model.train()
    print("Done with predictions")


class MyDataset(Dataset):
    def __init__(self, csv_file):
        self.csv = pd.read_csv(csv_file)

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        example = self.csv.iloc[index, :]
        features = example.iloc[: example.shape[0] - 4].to_numpy().astype(np.float32)
        labels = example.iloc[-4:-2].to_numpy().astype(np.int64)
        filenames = example.iloc[-2:].values.tolist()
        return features, labels, filenames


class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d((1536 + 1) * 2),
            nn.Linear((1536+1) * 2, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(500, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(100, 2),
        )

    def forward(self, x):
        return self.model(x)

=> Loading checkpoint


<ipython-input-22-35b5e6bda93c>:17: UserWarning: Important to have shuffle=False (and to ensure batch size is even size) when running get_csv_for_blend also set val_transforms to train_loader!
  warnings.warn("Important to have shuffle=False (and to ensure batch size is even size) when running get_csv_for_blend also set val_transforms to train_loader!")
 40%|████      | 427/1060 [08:31<33:12,  3.15s/it]

In [ ]:
model = MyModel().to(DEVICE)
ds = MyDataset(csv_file="train/train_blend.csv")
loader = DataLoader(ds, batch_size=256, num_workers=3, pin_memory=True, shuffle=True)
ds_val = MyDataset(csv_file="train/val_blend.csv")
loader_val = DataLoader(
    ds_val, batch_size=256, num_workers=3, pin_memory=True, shuffle=True
)
ds_test = MyDataset(csv_file="train/test_blend.csv")
loader_test = DataLoader(
    ds_test, batch_size=256, num_workers=2, pin_memory=True, shuffle=False
)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_fn = nn.MSELoss()

if LOAD_MODEL and "linear.pth.tar" in os.listdir():
    load_checkpoint(torch.load("linear.pth.tar"), model, optimizer, lr=1e-4)
    model.train()

for _ in range(5):
    losses = []
    for x, y, files in tqdm(loader_val):
        x = x.to(DEVICE).float()
        y = y.to(DEVICE).view(-1).float()

        # forward
        scores = model(x).view(-1)
        loss = loss_fn(scores, y)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Loss: {sum(losses)/len(losses)}")

if SAVE_MODEL:
    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint, filename="linear.pth.tar")

preds, labels = check_accuracy(loader_val, model)
print(cohen_kappa_score(labels, preds, weights="quadratic"))

preds, labels = check_accuracy(loader, model)
print(cohen_kappa_score(labels, preds, weights="quadratic"))

make_prediction(model, loader_test, "test_preds.csv")